In [ ]:
import os
import torch

from Utils.argparsing import get_args
from Preprocessing.data_init import organized_data_download
from Preprocessing.data_loading import BratsDataset

if __name__ == "__main__":

    data = BratsDataset()

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
import numpy as np

In [ ]:
rus = RandomUnderSampler(sampling_strategy=1, random_state=42)
x_res, y_res = rus.fit_resample(data.x, data.y)

zeros = 0
ones = 0

for val in y_res:
    if val==0:
        zeros+=1
    if val==1:
        ones+=1      

print(zeros)
print(ones)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_res, y_res, test_size=0.9, random_state=42)

In [ ]:
svc = SVC(random_state=42)
svc.fit(x_train, y_train)
predictions = svc.predict(x_test)
accuracy = svc.score(x_test, y_test)
print("SVC Test Accuracy: {:.4f}".format(accuracy*100))

In [ ]:
zeros = 0
ones = 0

for val in predictions:
    if val==0:
        zeros+=1
    if val==1:
        ones+=1     

print(zeros)
print(ones)

In [ ]:
y_test_trans = np.transpose(y_test)
dice = (2*np.matmul(y_test_trans, predictions)) / (np.sum(y_test + predictions))
print('Dice coefficient is %.6f' % dice)